In [ ]:
!pip install --upgrade pip setuptools wheel --quiet


In [ ]:
!pip install jedi --quiet


In [ ]:
!pip install datasets==2.7.1 --quiet



In [ ]:
!pip install pandas numpy scikit-learn --quiet


In [ ]:
from huggingface_hub import hf_hub_download
import pandas as pd

# Download the Parquet file manually
file_path = hf_hub_download(
    repo_id="IbrahimAmin/egyptian-arabic-hate-speech",
    filename="data/train-00000-of-00001.parquet",
    repo_type="dataset"
)

# Load using pandas
df = pd.read_parquet(file_path)
df.head()



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,text,label
0,انت في قمه النداله,Offensive
1,يلعن دين المسلمين ويلعن دين الحركه الاسلاميه ك...,Religious Discrimination
2,عيل زنجي عبد ابن كلب مشافش تربيه,Racism
3,كسم ليبيا والعراق واللي منهم إرهابيين ولاد وسخة,Racism
4,بضان بضان يعني مفيش كلام,Offensive


In [ ]:
print(df.columns)
print(df["label"].value_counts())


Index(['text', 'label'], dtype='object')
label
Neutral                     1708
Offensive                   1225
Racism                      1201
Sexism                      1201
Religious Discrimination    1200
Name: count, dtype: int64


In [ ]:
!pip install transformers --quiet



In [ ]:
from transformers import AutoTokenizer

# Load AraBERT tokenizer (use a popular pretrained arabic bert)
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02")

# Tokenize the text column, truncate to max_length 128 for example
tokens = tokenizer(
    list(df['text']),
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

print(tokens.keys())  # should show input_ids, attention_mask, etc.


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
label_mapping = {
    "Neutral": 0,
    "Offensive": 1,
    "Racism": 2,
    "Sexism": 3,
    "Religious Discrimination": 4
}

labels = df['label'].map(label_mapping).tolist()


In [ ]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = HateSpeechDataset(tokens, df['label'].tolist())


In [ ]:
from sklearn.model_selection import train_test_split

# Split indices and labels
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['label']  # keeps label distribution consistent
)

# Tokenize train and validation sets separately
train_encodings = tokenizer.batch_encode_plus(
    train_texts,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

val_encodings = tokenizer.batch_encode_plus(
    val_texts,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Create Dataset objects for train and validation sets
train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)


In [ ]:
import torch

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "aubmindlab/bert-base-arabertv2"
num_labels = len(set(df['label']))  # number of classes

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.optim import AdamW


In [ ]:
label2id = {
    "Neutral": 0,
    "Offensive": 1,
    "Racism": 2,
    "Sexism": 3,
    "Religious Discrimination": 4
}


In [ ]:
labels = [label2id[label] if isinstance(label, str) else label for label in labels]


In [ ]:
print(set(labels))  # see unique values and types


{0, 1, 2, 3, 4}


In [ ]:
# Get unique labels and assign integer IDs
unique_labels = df['label'].unique()
label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

# Map string labels to integers
df['label'] = df['label'].map(label2id)

# Confirm
print(label2id)
print(df['label'].value_counts())


{'Offensive': 0, 'Religious Discrimination': 1, 'Racism': 2, 'Sexism': 3, 'Neutral': 4}
label
4    1708
0    1225
2    1201
3    1201
1    1200
Name: count, dtype: int64


In [ ]:
!pip install farasapy

In [ ]:
from transformers import AutoTokenizer

# Load AraBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

# Tokenize all sentences
encodings = tokenizer(
    df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset

class HateSpeechDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels  # Already mapped to integers

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Integer labels
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
label2id = {label: i for i, label in enumerate(df['label'].unique())}
df['label_id'] = df['label'].map(label2id)
labels = df['label_id'].tolist()


In [ ]:
labels = df['label'].tolist()


In [ ]:
dataset = HateSpeechDataset(encodings, labels)


In [ ]:
from sklearn.model_selection import train_test_split

# Split encodings and labels using indices
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), labels, test_size=0.2, random_state=42
)

# Re-tokenize after splitting
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


In [ ]:
train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)


In [ ]:
from torch.utils.data import DataLoader

# Define batch size
batch_size = 16

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [ ]:
from transformers import BertForSequenceClassification

# Load model with number of labels
model = BertForSequenceClassification.from_pretrained(
    "aubmindlab/bert-base-arabertv2",
    num_labels=5  # Since we have 5 unique labels: 0–4
)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Loss function for classification
loss_fn = CrossEntropyLoss()


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch

# 1. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

# 2. Prepare labels and texts
texts = df['text'].tolist()
labels = df['label'].tolist()

# Map string labels to integers (if not done already)
label2id = {'Neutral':0, 'Offensive':1, 'Racism':2, 'Sexism':3, 'Religious Discrimination':4}

# 3. Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# 4. Tokenize function
def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

# 5. Custom Dataset class
class HateSpeechDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# 6. Create Dataset objects
train_dataset = HateSpeechDataset(train_texts, train_labels)
val_dataset = HateSpeechDataset(val_texts, val_labels)

# 7. Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

# Tokenize train and validation texts (batched)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


In [ ]:
import torch

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:
import torch
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [ ]:
small_train_dataset = torch.utils.data.Subset(train_dataset, list(range(1000)))
small_val_dataset = torch.utils.data.Subset(val_dataset, list(range(200)))

train_loader = DataLoader(small_train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(small_val_dataset, batch_size=8)


In [ ]:
epochs = 1


In [ ]:
from sklearn.metrics import classification_report

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=label2id.keys()))


                          precision    recall  f1-score   support

                 Neutral       0.81      0.61      0.69        41
               Offensive       0.91      0.80      0.85        40
                  Racism       0.77      0.88      0.82        26
                  Sexism       0.76      0.91      0.83        35
Religious Discrimination       0.82      0.88      0.85        58

                accuracy                           0.81       200
               macro avg       0.81      0.82      0.81       200
            weighted avg       0.82      0.81      0.81       200



In [ ]:
model.save_pretrained("arabic-hate-speech-model")
tokenizer.save_pretrained("arabic-hate-speech-model")


('arabic-hate-speech-model/tokenizer_config.json',
 'arabic-hate-speech-model/special_tokens_map.json',
 'arabic-hate-speech-model/vocab.txt',
 'arabic-hate-speech-model/added_tokens.json',
 'arabic-hate-speech-model/tokenizer.json')

In [ ]:
def predict_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    label_map = {v: k for k, v in label2id.items()}
    return label_map[prediction]


In [ ]:
predict_text("انت في قمه النداله")


'Neutral'

In [ ]:
import gradio as gr
import torch


# label2id = {'Neutral':0, 'Offensive':1, 'Racism':2, 'Sexism':3, 'Religious Discrimination':4}
id2label = {v: k for k, v in label2id.items()}

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    return id2label[prediction]


In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=3, placeholder="أدخل نصًا باللغة العربية..."),
    outputs="text",
    title="Arabic Hate Speech Classifier",
    description="أدخل جملة باللغة العربية وسنقوم بتصنيفها (محايد، عنصري، مسيء، إلخ)"
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15653c67142b28b734.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [58]:
import gradio as gr
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model_name = "aubmindlab/bert-base-arabertv2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

labels = ['Neutral', 'Offensive', 'Racism', 'Sexism', 'Religious Discrimination']

def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    conf, predicted = torch.max(probs, dim=1)
    label = labels[predicted.item()]
    return {label: conf.item()}

iface = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(lines=2, placeholder="Enter Arabic text here..."),
    outputs=gr.Label(num_top_classes=3),
    title="Arabic Text Classifier",
    description="Classify Arabic text into Neutral, Offensive, Racism, Sexism, or Religious Discrimination."
)

if __name__ == "__main__":
    iface.launch()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0034907e95442c5d13.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
